# 

# Model Descriptions

## Overview

In this analysis, I compare the GARCH(1,1) model to the Stochastic Volatility (SV) model in order to forecast the variance of the return series. This is of interest given that while there is low autocorrelation in the return of financial time series, the volatility of returns has been observed to have persistent autocorrelation. Therefore I will seek to evaluate the goodness-of-fit of these two alternate approaches. 

Crucially, the GARCH model calculates the variance of returns deterministically, whereas the stochastic volatility model treats it as a random variable.

# Stochastic Volatility (SV) Model

**Model Specification**

*De-Meaned Returns $y_t$:*

$$
y_t = e^{\frac{h_t}{2}} \cdot \epsilon_t
$$

*Log-Volatility $h_t$:*
$$ 
h_t = \mu + \phi(h_{t-1} - \mu) + \sigma \cdot \eta_t
$$


$\epsilon_t$ and $\eta_t$ are gaussian white noise:

$$
\epsilon_t \sim \mathcal{N}(0,1)
$$
$$
\eta_t \sim \mathcal{N}(0,1)
$$

**Parameters**

$\phi \in (-1, 1)$

$ \sigma > 0 $


**Priors**

$\mathcal{P}(\mu) \sim \mathcal{N}(-0.5, 1)$

$\mathcal{P}(\phi) \sim \mathcal{N}(0.75, 0.1^2) \cdot \mathbb{1}_{[-1, 1]}$ 

$\mathcal{P}(\sigma) \sim IG(2, 0.5)$




# GARCH(1,1) Model 

**Model Specification**

*De-Meaned Returns $y_t$:*

$$
y_t = \epsilon_t
$$

*Conditional Variance Equation (GARCH(1,1)):*
$$
\sigma_t^2 = \mu + \alpha_1 \epsilon_{t-1}^2 + \beta_1 \sigma_{t-1}^2
$$


$\epsilon_t$ is a gaussian error term:

$$
\epsilon_t \sim \mathcal{N}(0,\sigma_t^2)
$$


**Parameters:**
- $\mu > 0$ ensures non-negative variance.
- $\alpha_1, \beta_1 \geq 0$ and $\alpha_1 + \beta_1 < 1$ for stationarity.


**Priors**

$\mu \sim IG(1.5, 0.25)$

$\alpha_1 \sim \beta(2, 5)$ 

$\beta_1 \sim \beta(5, 2)$




